In [1]:
# import dependencies
import configparser
import h5py
import json
import numpy as np
import os
import pathlib
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import shutil
import tensorflow as tf
import utils
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
import snntoolbox.bin.run as snn

In [2]:
# create datasets
data_path = pathlib.Path(r'../datasets/RAFDB.npz')
with np.load(data_path) as data:
    images = data['images']
    labels = data['labels']
    classes = data['classes']
print(f"Found {images.shape[0]} examples in {labels.shape[1]} classes.")
print(f"Shape= {(images.shape[1], images.shape[2])}")
print(f"Class Names= {classes}")

tf.random.set_seed(1)

# constants
CLASS_LABELS = classes
NUM_CLASSES = len(classes)
BATCH_SIZE = 32
IMAGE_DIMS = (48, 48)
SEED = 123
_TRAIN = .7
_VAL = .15
_TEST = .1

# create a dataset object and shuffle
dataset = tf.data.Dataset.from_tensor_slices((images, labels))
dataset = dataset.shuffle(len(images) * BATCH_SIZE, seed=SEED)
shuffle_images = []
shuffle_labels = []
for im, l in dataset.as_numpy_iterator():
    shuffle_images.append(im)
    shuffle_labels.append(l)
shuffle_images = np.asarray(shuffle_images)
shuffle_labels = np.asarray(shuffle_labels)

# create the slicing indices
TRAIN = round(len(dataset) * _TRAIN)
VAL = TRAIN + round(len(dataset) * _VAL)
TEST = VAL + round(len(dataset) * _TEST)

train_ds = utils.data_pipeline(
    shuffle_images[0:TRAIN], shuffle_labels[0:TRAIN], IMAGE_DIMS,
    edges=False, batch_size=BATCH_SIZE, flip=False, haar=False)
val_ds = utils.data_pipeline(
    shuffle_images[TRAIN:VAL], shuffle_labels[TRAIN:VAL], IMAGE_DIMS,
    edges=False, batch_size=BATCH_SIZE, flip=False, haar=False)
test_ds = utils.data_pipeline(
    shuffle_images[VAL:TEST], shuffle_labels[VAL:TEST], IMAGE_DIMS,
    edges=False, batch_size=BATCH_SIZE, flip=False, haar=False)


print(f"Using {len(train_ds)*BATCH_SIZE} samples for training.")
print(f"Using {len(val_ds)*BATCH_SIZE} samples for validation.")
print(f"Using {len(test_ds)*BATCH_SIZE} samples for testing.")

Found 15339 examples in 7 classes.
Shape= (48, 48)
Class Names= ['Surprise' 'Fear' 'Disgust' 'Happiness' 'Sadness' 'Anger' 'Neutral']
Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-03-06 23:17:56.421292: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-06 23:17:56.421400: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


--------------------------------------------------------------------------------

Dataset Info:
---> size: 24.73805 mb

---> dimensions: (48, 48)
---> samples: 10737
---> batches: 335

Expanding image dimensions...
---> elapsed time: 17.03762 ms

Dataset size after reshaping:
---> size: 24.73805 mb

Creating dataset object..

Batch shapes: 
(335, 32, 48, 48, 1) (335, 32, 7)
---> elapsed time: 11507.73179 ms

Setting batch size to 32
---> elapsed time: 1.86892 ms

Configuring prefetching...
---> elapsed time: 0.73613 ms

Scaling images...
---> elapsed time: 85.25567 ms

Configuring caching...
---> elapsed time: 0.18029 ms

Total time ===> 11.612866624999999 s
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------

Dataset Info:
---> size: 5.3015 mb

---> dimensions: (48, 48)
---> samples: 2301
---> batches: 71

Expanding image dimensions...
---> elapsed time: 3.60783 ms

Dataset si

In [3]:
# store the data for SNN conversion/simulation
x_test = []
y_test = []
x_norm = []

# create the dataset for SNN simulation
for im, l in test_ds.as_numpy_iterator():
    for i in range(np.shape(im)[0]):
        #print(im[i, :, :, :].shape)
        x_test.append(im[i, :, :, :])
        y_test.append(l[i, :])

# create a dataset for normalization
for im, l in list(train_ds.as_numpy_iterator()):
    for i in range(np.shape(im)[0]):
        #print(im[i, :, :, :].shape)
        x_norm.append(im[i, :, :, :])

x_test = np.asarray(x_test)
y_test = np.asarray(y_test)
x_norm = np.asarray(x_norm)
### print(x_test.shape)
### print(y_test.shape)

### x_test = np.load(os.path.join('./data', 'x_test.npy'))
### y_test = np.load(os.path.join('./data', 'y_test.npy'))
### 
### print(f"X_Data shape ---> {x_test.shape}")
### print(f"Y_Data shape ---> {y_test.shape}")
### 
### np.savez_compressed(r'./data/x_test', x_test)
### np.savez_compressed(r'./data/y_test', y_test)
### np.savez_compressed(r'./data/x_norm', x_norm)

2023-03-06 23:18:12.996876: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [4]:
model_name = "VGG_b3k112k224k344k460u185u2100p1_0.16p2_0.21"
test_model = tf.keras.models.load_model(f'./models/{model_name}.h5')
test_model.summary()

train_new = True

if train_new == False:
    model = test_model

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 48, 48, 12)        1476      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 12)       0         
 )                                                               
                                                                 
 sequential_2 (Sequential)   (None, 24, 24, 24)        7920      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 24)       0         
 2D)                                                             
                                                                 
 sequential_3 (Sequential)   (None, 12, 12, 44)        27192     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 6, 44)         0

In [5]:
## tf.keras.utils.plot_model(
##     model,
##     to_file='./out/test_architecture.png',
##     show_shapes=True,
##     show_dtype=False,
##     show_layer_names=True,
##     rankdir='TB',
##     expand_nested=False,
##     dpi=150,
##     layer_range=None,
##     show_layer_activations=True
## )


for layer in test_model.layers[6:]:
    print(layer)

In [6]:
if train_new:
    # build model
    model = tf.keras.models.Sequential([

        tf.keras.layers.Input(shape=(48, 48, 1)),

        tf.keras.layers.Conv2D(filters=12, kernel_size=(3, 3), padding="same", activation="relu"),
        tf.keras.layers.Conv2D(filters=12, kernel_size=(3, 3), padding="same", activation="relu"),
        #tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2)),


        tf.keras.layers.Conv2D(filters=24, kernel_size=(3, 3), padding="same", activation="relu"),
        tf.keras.layers.Conv2D(filters=24, kernel_size=(3, 3), padding="same", activation="relu"),
        #tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2)),

        tf.keras.layers.Conv2D(filters=44, kernel_size=(3, 3), padding="same", activation="relu"),
        tf.keras.layers.Conv2D(filters=44, kernel_size=(3, 3), padding="same", activation="relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2)),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(85, activation = 'relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(100, activation = 'relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(7, activation = 'softmax'),

    ])
    ## for layer in test_model.layers[6:]:
    ##     print(f"Adding layer {layer.name}")
    ##     model.add(layer)

    model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 12)        120       
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 12)        1308      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 12)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 24, 24, 24)        2616      
                                                                 
 conv2d_3 (Conv2D)           (None, 24, 24, 24)        5208      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 24)       0         
 2D)                                                    

In [7]:
if train_new == False:
    # copy Sequential_1
    for i in range(0, 3):
        print(f"Copying weights from {test_model.layers[0].layers[i].name} ---> {model.layers[i].name}")
        model.layers[i].set_weights(test_model.layers[0].layers[i].get_weights())

    # copy max pooling 1
    print(f"Copying weights from {test_model.layers[1].name} ---> {model.layers[3].name}")
    model.layers[3].set_weights(test_model.layers[1].get_weights())

    # copy Sequential_2
    for i in range(4, 7):
        print(f"Copying weights from {test_model.layers[2].layers[i - 4].name} ---> {model.layers[i].name}")
        model.layers[i].set_weights(test_model.layers[2].layers[i - 4].get_weights())

    # copy max pooling 2
    print(f"Copying weights from {test_model.layers[3].name} ---> {model.layers[7].name}")
    model.layers[7].set_weights(test_model.layers[3].get_weights())

    # copy Sequential_3
    for i in range(8, 11):
        print(f"Copying weights from {test_model.layers[4].layers[i - 8].name} ---> {model.layers[i].name}")
        model.layers[i].set_weights(test_model.layers[4].layers[i - 8].get_weights())

    # copy max pooling 3
    print(f"Copying weights from {test_model.layers[5].name} ---> {model.layers[7].name}")
    model.layers[11].set_weights(test_model.layers[5].get_weights())


    # copy remaining layers
    print("Remaing layers -->")
    for i in range(12, len(model.layers)):
        print(f"Copying weights from {test_model.layers[i - 6].name} ---> {model.layers[i].name}")
        model.layers[i].set_weights(test_model.layers[i - 6].get_weights())



In [8]:

model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.005),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

## if os.path.exists(f'./out/{model_name}.h5'): os.system(f"rm ./out/{model_name}.h5")
## if os.path.exists(f'./out/{model_name}_INI.h5'): os.system(f"rm ./out/{model_name}_INI.h5")
## if os.path.exists(f'./out/{model_name}_parsed.h5'): os.system(f"rm ./out/{model_name}_parsed.h5")

# add checkpoint callback
checkpoint = tf.keras.callbacks.ModelCheckpoint(
                f'./out/{model_name}.h5',
                monitor='val_accuracy',
                verbose=2,
                save_best=True,
                save_weights_only=False,
                mode='max')

# add early stopping callback
print("\nConfiguring early stopping parameters.")
early_stop = tf.keras.callbacks.EarlyStopping(
                monitor='val_loss',
                min_delta=.005,
                patience=5,
                verbose=2,
                mode='auto')

hist = model.fit(
                train_ds,
                validation_data=val_ds,
                epochs=25,
                verbose=1,
                callbacks=[checkpoint])

#loss, acc = model.evaluate(x=x_test, y=y_test, verbose=1)
m = tf.keras.models.load_model(f'./out/{model_name}.h5')
loss, acc = m.evaluate(x_test, y_test, verbose=1)
print(f"loss ---> {loss}\naccuracy ---> {acc}")


Configuring early stopping parameters.
Epoch 1/25


2023-03-06 23:18:14.064622: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


336/336 [==============================] - ETA: 0s - loss: 1.6345 - accuracy: 0.3904

2023-03-06 23:18:20.655481: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.



Epoch 1: saving model to ./out/VGG_b3k112k224k344k460u185u2100p1_0.16p2_0.21.h5
336/336 [==============================] - 8s 21ms/step - loss: 1.6345 - accuracy: 0.3904 - val_loss: 1.7822 - val_accuracy: 0.4363
Epoch 2/25
335/336 [============================>.] - ETA: 0s - loss: 1.3679 - accuracy: 0.5007
Epoch 2: saving model to ./out/VGG_b3k112k224k344k460u185u2100p1_0.16p2_0.21.h5
336/336 [==============================] - 8s 22ms/step - loss: 1.3672 - accuracy: 0.5009 - val_loss: 1.2646 - val_accuracy: 0.5602
Epoch 3/25
334/336 [============================>.] - ETA: 0s - loss: 1.2001 - accuracy: 0.5672
Epoch 3: saving model to ./out/VGG_b3k112k224k344k460u185u2100p1_0.16p2_0.21.h5
336/336 [==============================] - 7s 22ms/step - loss: 1.1995 - accuracy: 0.5676 - val_loss: 1.1068 - val_accuracy: 0.6093
Epoch 4/25
335/336 [============================>.] - ETA: 0s - loss: 1.1049 - accuracy: 0.6043
Epoch 4: saving model to ./out/VGG_b3k112k224k344k460u185u2100p1_0.16p2_0.2

2023-03-06 23:21:13.619811: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


48/48 [==============================] - 1s 11ms/step - loss: 1.3522 - accuracy: 0.6343
loss ---> 1.3522313833236694
accuracy ---> 0.6342894434928894


In [9]:
# create config file for snntoolbox
config = configparser.ConfigParser()

# set up data/output paths
config['paths'] = {
    'path_wd': './out',
    'dataset_path': './data',
    'runlabel': model_name,
    'filename_ann': model_name
}

# configure tools
config['tools'] = {
    'evaluate_ann': False,
    'parse': True,
    'normalize': True,
    'simulate': True
}

# configure conversion parameters
config['conversion'] = {
    'max2avg_pool': True
}

# configure simulation settings
config['simulation'] = {
    'simulator': 'INI',
    'duration': 64,
    'batch_size': 24,
    'num_to_test': 120,
    'keras_backend': 'tensorflow'
}

# configure the cell parameters
config['cell'] = {
    'reset': """'Reset by subtraction'"""
}

# configure output parameters
config['output'] = {
    'plot_vars': {
        # 'input_image',
        # 'spiketrains',
        # 'spikerates',
        # 'spikecounts',
        # 'operations',
        # 'normalization_activations',
        # 'activations',
        # 'correlation',
        # 'v_mem',
        # 'error_t'
    },
    'verbose': 0,
    'overwrite': True
}

with open('./config.ini', 'w') as configfile:
    config.write(configfile)

# run snn conversion/simulation
#snn.main("./config.ini")
